In [299]:
def start():
    !pip install unidecode
    !pip install wordcloud
    
    import unidecode
    import urllib.request
    import pickle
    import pandas as pd
    import numpy as np
    import nltk
    import string
    
    import matplotlib.pyplot as plt
    from wordcloud import WordCloud

In [301]:
!pip install wordcloud

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Faradezo\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Faradezo\\AppData\\Local\\Temp\\pip-install-cg_7qwzv\\wordcloud_392b5bbcc9ed4fe5a30e7b07eef6492c\\setup.py'"'"'; __file__='"'"'C:\\Users\\Faradezo\\AppData\\Local\\Temp\\pip-install-cg_7qwzv\\wordcloud_392b5bbcc9ed4fe5a30e7b07eef6492c\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Faradezo\AppData\Local\Temp\pip-wheel-i6d3yrw7'
       cwd: C:\Users\Faradezo\AppData\Local\Temp\pip-install-cg_7qwzv\wordcloud_392b5bbcc9ed4fe5a30e7b07eef6492c\
  Complete output (20 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\li


  Using cached wordcloud-1.8.1.tar.gz (220 kB)
  Running setup.py clean for wordcloud
Failed to build wordcloud
    Running setup.py install for wordcloud: started
    Running setup.py install for wordcloud: finished with status 'error'


    copying wordcloud\__init__.py -> build\lib.win-amd64-3.9\wordcloud
    copying wordcloud\__main__.py -> build\lib.win-amd64-3.9\wordcloud
    copying wordcloud\stopwords -> build\lib.win-amd64-3.9\wordcloud
    copying wordcloud\DroidSansMono.ttf -> build\lib.win-amd64-3.9\wordcloud
    UPDATING build\lib.win-amd64-3.9\wordcloud/_version.py
    set build\lib.win-amd64-3.9\wordcloud/_version.py to '1.8.1'
    running build_ext
    building 'wordcloud.query_integral_image' extension
    error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
    ----------------------------------------
ERROR: Command errored out with exit status 1: 'C:\Users\Faradezo\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Faradezo\\AppData\\Local\\Temp\\pip-install-cg_7qwzv\\wordcloud_392b5bbcc9ed4fe5a30e7b07eef6492c\\setup.py'"'"'; __file__='"'"'C:\\Users\

In [270]:
def inicia_base():
    link = 'https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/sdp.pkl'
    return pd.read_pickle(link)#ref

In [271]:
def trata_texto(texto_para_verificacao):
    texto = []
    for line in urllib.request.urlopen(texto_para_verificacao):
        texto.append(line.decode('utf-8'))
        
    nltk.download('stopwords')                              # baixa arquivo com as stopwords
    stopwords = nltk.corpus.stopwords.words('portuguese')   #seleciona a tabela de palavras em portugues
    
    for i in range(len(texto)):
        for chars in string.punctuation:
            texto[i] = texto[i].replace(chars,'') # !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~
            texto[i] = texto[i].replace('—','')   # este simbolo nao esta incluso no strings porem existe no documento
            texto[i] = texto[i].replace('ª','')
            texto[i] = texto[i].replace('º','')
            texto[i] = unidecode.unidecode(texto[i]) #remove acentos
    
    for i in range(len(texto)): #remove numeros
        for chars in string.digits:
            texto[i] = texto[i].replace(chars,'')
    
    mydict = {}                             # cria um dicionário vazio
    
    for line in texto:
        line = line.lower()                    # converte para lower
        words = line.split()                   # separa cada palavra
        for word in words:                     # para cada palavra em words
            if word not in stopwords:          # exclui palavras genericas
                if word not in mydict.keys():  # se palavra não está no dicionário
                    mydict[word] = 1           # acrescenta a word com o valor 1
                else:                          # se a entrada já existe
                    mydict[word] += 1          # soma 1 ao valor já existente
                    
    a = pd.DataFrame.from_dict(mydict, columns=['presente'], orient = 'index')
    a = a.sort_values(by=['presente'], ascending=False)
    return a

In [280]:
def comparador(ref, checar):
    n_words = 30
    resultados = {}
    for col in ref.keys():
        original = pd.DataFrame(ref.get(col), columns=['presente'])
        pd_comp = pd.merge(left=original, right=checar, how='left', left_index=True, right_index=True)
        pd_comp.replace(np.nan, 0, inplace=True)
        pd_compT = pd_comp.T
        
        pd_compT['mais_comuns'] = (pd_compT.ne(0).sum(axis=1)) / n_words
        #pd_compT.drop(pd_compT.columns[0:30], axis=1, inplace=True)
        pd_compT.T.round(2)
        resultados[col] = pd_compT['mais_comuns'][1]
    return resultados

In [252]:
ref.keys()
#ref.get('alexandre_garcia')'Reinaldo_Azevedo'
b = pd.DataFrame(ref.get('alexandre_garcia'), columns=['presente'])

In [264]:
pd_compT['mais_comuns'][1]

0.3333333333333333

In [258]:
n_words = 30

pd_comp = pd.merge(left=b, right=a, how='left', left_index=True, right_index=True)
   
pd_comp.replace(np.nan, 0, inplace=True)
pd_compT = pd_comp.T

pd_compT['mais_comuns'] = (pd_compT.ne(0).sum(axis=1)) / n_words
#pd_compT.drop(pd_compT.columns[0:30], axis=1, inplace=True)
pd_compT.T.round(2)

,presente_x,presente_y
presidente,0.64,7.00
pais,0.55,1.00
anos,0.55,0.00
ainda,0.52,2.00
ser,0.52,0.00
agora,0.51,0.00
porque,0.49,0.00
sobre,0.47,2.00
vai,0.44,0.00
brasil,0.44,0.00


# Utilizar apenas os itens abaixo

o texto para verificacao deve ser um texto salvo em txt

In [276]:
texto_para_verificacao = 'https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/verificar/texto1.txt'

In [300]:
start()
ref = inicia_base()
check = trata_texto(texto_para_verificacao)
results = comparador(ref, check)

  Using cached wordcloud-1.8.1.tar.gz (220 kB)
  Running setup.py clean for wordcloud
Failed to build wordcloud
    Running setup.py install for wordcloud: started
    Running setup.py install for wordcloud: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Faradezo\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Faradezo\\AppData\\Local\\Temp\\pip-install-pcvmhhf1\\wordcloud_f919f1660130439c842f6540f41a921b\\setup.py'"'"'; __file__='"'"'C:\\Users\\Faradezo\\AppData\\Local\\Temp\\pip-install-pcvmhhf1\\wordcloud_f919f1660130439c842f6540f41a921b\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Faradezo\AppData\Local\Temp\pip-wheel-3vopq6xj'
       cwd: C:\Users\Faradezo\AppData\Local\Temp\pip-install-pcvmhhf1\wordcloud_f919f1660130439c842f6540f41a921b\
  Complete output (20 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\li

ModuleNotFoundError: No module named 'wordcloud'

In [287]:
dict(sorted(results.items(), key=lambda item: item[1], reverse=True) )

{'Reinaldo_Azevedo': 0.36666666666666664,
 'Thays_Oyama': 0.36666666666666664,
 'Walter_Maierovitch': 0.36666666666666664,
 'alexandre_garcia': 0.3333333333333333,
 'Josias_de_Souza': 0.3333333333333333,
 'Carla_Araujo': 0.3,
 'Kennedy_Alencar': 0.3,
 'Leonardo_Sakamoto': 0.3,
 'Ricardo_Kotscho': 0.26666666666666666,
 'Tales_Faria': 0.26666666666666666,
 'arnaldo_jabor': 0.23333333333333334,
 'Juca_Kfouri': 0.23333333333333334,
 'Mauricio_Stycer': 0.16666666666666666}

In [298]:
#word_could_dict = {'Git':100, 'GitHub':100, 'push':50, 'pull':10, 'commit':80, 'add':30, 'diff':10, 'mv':5, 'log':8, 'branch':30, 'checkout':25}

wordcloud = WordCloud(width = 1000, height = 500).generate_from_frequencies(results)

plt.figure(figsize=(15,8))
plt.imshow(wordcloud)

  Running setup.py clean for wordcloud
Failed to build wordcloud
    Running setup.py install for wordcloud: started
    Running setup.py install for wordcloud: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Faradezo\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Faradezo\\AppData\\Local\\Temp\\pip-install-uhhngjf0\\wordcloud_7264e65cf3e24ad8a79969554d651dd7\\setup.py'"'"'; __file__='"'"'C:\\Users\\Faradezo\\AppData\\Local\\Temp\\pip-install-uhhngjf0\\wordcloud_7264e65cf3e24ad8a79969554d651dd7\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Faradezo\AppData\Local\Temp\pip-wheel-ckfdo1hk'
       cwd: C:\Users\Faradezo\AppData\Local\Temp\pip-install-uhhngjf0\wordcloud_7264e65cf3e24ad8a79969554d651dd7\
  Complete output (20 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\li

ModuleNotFoundError: No module named 'wordcloud'